### Ingestión del archivo language

Paso 1 - Leer el archivo csv usando "DataFrameReader" de spark

In [0]:
dbutils.widgets.text('p_enviroment', '')
v_enviroment = dbutils.widgets.get('p_enviroment')

In [0]:
dbutils.widgets.text('p_file_date', '2024-12-16')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import *

In [0]:
language_schema = StructType([
    StructField('languageId', IntegerType(), False),
    StructField('languageCode', StringType(), True),
    StructField('languageName', StringType(), True)
])

In [0]:
language_df = spark.read \
    .option("header", True) \
    .schema(language_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/language.csv")

### Paso 2 - Seleccionar las columnas "requeridas"

In [0]:
from pyspark.sql.functions import col

In [0]:
language_selected_df = language_df.select(col("languageId"), col("languageName"))

### Paso 3 - Cambiar el nombre de las columnas según lo requerido

In [0]:
languague_renamed_df = language_selected_df.withColumnsRenamed({"languageId": "language_id", "languageName": "language_name"})

### Paso 4 - Agregar la columna "ingestion_date" y "env" al DataFrame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
languague_final_df = add_ingestion_date(languague_renamed_df) \
    .withColumn("enviroment", lit(v_enviroment)) \
    .withColumn("file_date", lit(v_file_date))

### Paso 5 - Escribir datos en el DeltaLake en formato "Parquet"

In [0]:
languague_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.languages
GROUP BY file_date;

In [0]:
display(spark.read.format('delta').load(f"{silver_folder_path}/languages"))

In [0]:
%sql
SELECT * FROM movie_silver.languages;

In [0]:
dbutils.notebook.exit("Success")